In [3]:
# !pip install openai==0.28
from openai import OpenAI
import re
import httpx
import os
import json
from getpass import getpass
import requests
from bs4 import BeautifulSoup

try:
    os.chdir('..')
    from lib.scraper import WikiScraper
finally:
    os.chdir('test_notebooks/')

openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
# wiki search interfaces with article

# interface??

# ReAct uses CoT and must use the article

# Maybe: input a question and output paragraph(s) of related header(s) --> use the same (or another) LLM to determine which headers are related
# Maybe: input keywords and output matching paragraphs --> use another LLM to determine what keywords would be good to search in the wiki article

In [4]:
class Actions:
    
    def __init__(self):
        self.scraper = WikiScraper()
        
    def query_wikipedia(self, topic, question):
        """Search wikipedia with a general topic and a specific question"""
        raise NotImplementedError()
    
    def query_user(self, question):
        """Query the user with a clarification question"""
        return input(question)
    
    def query_google(self, prompt):
        """Query google for less reliable but potentially more diverse results"""
        raise NotImplementedError()

In [ ]:
class ActionLLM(Actions):
    
    def __init__(self, model="gpt-3.5-turbo"):
        super().__init__()
        self.client = OpenAI()
        self.model = model
        
    def search_wikipedia(self, topic, question):
        self.scraper.set_topic(topic)
        headers = self.scraper.get_headers()
        related_headers = None # TODO: ask a diff LLM which ones are related
        ret = []
        for header in related_headers:
            ret.append(self.scraper.get_subsection(header=header))
        
        # provide summary of these? for smaller context windows
        
        return ret
    
    def query_user(self, question):
        return super().query_user(question)
    
    def query_google(self, prompt):
        return super().query_google(prompt)
        

**Question**: Who almost always wins the soccer world cup?

**Thought**: The user wants to know more about the soccer world cup and the team that usually wins. Wikipedia might be a good source for this. They likely have an article related to the soccer world cup. A specific question I could ask is "Which team has won the world cup?"

**Action**: `WikiQuery['soccer world cup', 'Which team has won the world cup']`

**Result**: In all, 80 nations have played in at least one World Cup.[a] Of these, eight national teams have won the World Cup,[115] and they have added stars to their badges, with each star representing a World Cup victory. Uruguay, however, chose to display four stars on their badge, representing their two gold medals at the 1924 and 1928 Summer Olympics, which are recognized by FIFA as World Championships, and their two World Cup titles in 1930 and 1950.

With five titles, Brazil are the most successful World Cup team and also the only nation to have played in every World Cup (22) to date.[116] Brazil were also the first team to win the World Cup for the third (1970), fourth (1994) and fifth (2002) time. Italy (1934 and 1938) and Brazil (1958 and 1962) are the only nations to have won consecutive titles. West Germany (1982–1990) and Brazil (1994–2002) are the only nations to appear in three consecutive World Cup finals. Germany has made the most top-four finishes (13), medals (12), as well as the most finals (8).

To date, the final of the World Cup has only been contested by teams from the UEFA (Europe) and CONMEBOL (South America) confederations. European nations have won twelve titles, while South American nations have won ten. Only three teams from outside these two continents have ever reached the semi-finals of the competition: United States (North, Central America and Caribbean) in 1930; South Korea (Asia) in 2002; and Morocco (Africa) in 2022. Only one Oceanian qualifier, Australia in 2006, has advanced to the second round, a feat they later reaccomplished in 2022.[b]

Brazil, Argentina, Spain and Germany are the only teams to win a World Cup hosted outside their continental confederation; Brazil came out victorious in Europe (1958), North America (1970 and 1994) and Asia (2002). Argentina won a World Cup in North America in 1986 and in Asia in 2022. Spain won in Africa in 2010. In 2014, Germany became the first European team to win in the Americas. Only on five occasions have consecutive World Cups been won by teams from the same continent; the longest streak of tournaments won by a single confederation is four, with the 2006, 2010, 2014, and 2018 tournaments all won by UEFA teams (Italy, Spain, Germany, and France, respectively).

**Observation**: The second paragraph notes that Brazil is the most successful World Cup team. This is very related to the question asked by the user. I should clarify my current answer matches what they want

**Action**: `UserQuery['Are you looking for the most successful FIFA World cup team?']`

**Result**: Yes.

**Observation**: The user confirmed my thought process, so I can answer now.

**Action**: `Answer['Brazil is the most successful World Cup team.']`